In [1]:
from data.process import *

2024-07-23 19:25:41.688339: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-23 19:25:41.695810: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-23 19:25:41.706884: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-23 19:25:41.706913: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-23 19:25:41.714058: I tensorflow/core/platform/cpu_feature_gua

In [2]:
train_df, dev_df, test_df = retrieve_train_dev_test_as_category_for_xgboost()
X_train, Y_train = split_input_output(train_df)
X_dev, Y_dev = split_input_output(dev_df)

In [3]:
from lightgbm import LGBMClassifier, log_evaluation
from model.lgbm_opt import *

In [4]:
lgbm_params = {
    **lgbm_fixed_params,
    # categorical features
    'enable_categorical': True,
    # fixed after secone optimization
    'colsample_bytree': 0.21039136022674532,
    'learning_rate': 0.1,
    'max_depth': 25,
    'num_leaves': 221,
    # override fixed params
    'n_estimators': 10000,
    'early_stopping_rounds': 200,
}

In [5]:
cat_features = list(X_train.select_dtypes(include=['object']).columns)

In [6]:
model = LGBMClassifier(
    **lgbm_params,
)
log_callback = log_evaluation(period=100)
boosting = model.fit(X_train, Y_train, eval_set=[(X_dev, Y_dev)], categorical_feature=cat_features, eval_metric='auc', callbacks=[log_callback])

[100]	valid_0's auc: 0.880251	valid_0's binary_logloss: 0.251905
[200]	valid_0's auc: 0.882322	valid_0's binary_logloss: 0.249134
[300]	valid_0's auc: 0.883508	valid_0's binary_logloss: 0.248035
[400]	valid_0's auc: 0.884198	valid_0's binary_logloss: 0.247428
[500]	valid_0's auc: 0.885005	valid_0's binary_logloss: 0.246828
[600]	valid_0's auc: 0.885657	valid_0's binary_logloss: 0.246361
[700]	valid_0's auc: 0.886172	valid_0's binary_logloss: 0.246004
[800]	valid_0's auc: 0.886459	valid_0's binary_logloss: 0.245757
[900]	valid_0's auc: 0.886906	valid_0's binary_logloss: 0.245428
[1000]	valid_0's auc: 0.887234	valid_0's binary_logloss: 0.245187
[1100]	valid_0's auc: 0.887398	valid_0's binary_logloss: 0.245064
[1200]	valid_0's auc: 0.88785	valid_0's binary_logloss: 0.244716
[1300]	valid_0's auc: 0.887911	valid_0's binary_logloss: 0.244682
[1400]	valid_0's auc: 0.888064	valid_0's binary_logloss: 0.244559
[1500]	valid_0's auc: 0.888274	valid_0's binary_logloss: 0.244388
[1600]	valid_0's auc

In [7]:
import pprint
pprint.pprint(model.best_score_)

defaultdict(<class 'collections.OrderedDict'>,
            {'valid_0': OrderedDict([('auc', 0.8899780621320712),
                                     ('binary_logloss', 0.24309779532520515)])})


In [8]:
lgbm_output = model.predict_proba(test_df, iteration_range=(0, model._best_iteration + 1))[:, 1]